# Novelty widget

Loads with default (x, sr). Params are live or Recompute. Output can be used by **tempogram_widget.ipynb** (set `NOVELTY_FOR_TEMPOGRAM` there or run this notebook first and use defaults).

In [1]:
import sys
from pathlib import Path

import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML, Javascript, display

widget_dir = Path.cwd()
for candidate in [Path.cwd(), Path.cwd() / "widget", Path.cwd() / "scratch" / "widget", Path.cwd() / "notebooks" / "scratch" / "widget"]:
    if (candidate / "temporal_widget_helpers.py").exists():
        widget_dir = candidate
        break
sys.path.insert(0, str(widget_dir))
project_root = widget_dir
for _ in range(6):
    if (project_root / "src" / "dijon").exists():
        break
    project_root = project_root.parent
sys.path.insert(0, str(project_root))

from temporal_widget_helpers import (
    build_audio_with_waveform_html,
    FS_NOV,
    GAMMA_OPTIONS,
    HOP_OPTIONS,
    LOCAL_M_OPTIONS,
    NOVELTY_DEFAULTS,
    eager_compute_novelty,
    get_novelty_for_method,
    load_audio_and_markers,
    zoom_level_to_half_sec,
)

%matplotlib inline

In [2]:
TRACK = "YTB-005"
MARKER = ""

x, sr, seg_start, seg_end, marker_label = load_audio_and_markers(TRACK, MARKER or None)
duration_s = len(x) / sr
print(f"Track: {TRACK} | Marker: {marker_label} | Duration: {duration_s:.2f}s @ {sr} Hz")

Track: YTB-005 | Marker: FULL | Duration: 175.48s @ 22050 Hz


In [3]:
novelty_cache, init_time, _ = eager_compute_novelty(x, sr)
print(f"Eager init: {init_time:.2f}s ({len(novelty_cache)} novelty variants)")

Eager init: 98.97s (896 novelty variants)


In [4]:
_compact = dict(layout=widgets.Layout(margin="0 2px 2px 2px"))

nov_w_method = widgets.Dropdown(options=["energy", "spectral", "phase", "complex"], value="spectral", description="Novelty:", **_compact)
nov_w_hop = widgets.Dropdown(options=HOP_OPTIONS, value=256, description="", **_compact)
nov_w_gamma = widgets.Dropdown(options=GAMMA_OPTIONS, value=100.0, description="", **_compact)
nov_w_local_avg = widgets.Dropdown(options=LOCAL_M_OPTIONS, value=10, description="", **_compact)
nov_w_zoom_enabled = widgets.Checkbox(value=False, description="Zoom on playhead", **_compact)
nov_w_zoom_level = widgets.IntSlider(value=1, min=1, max=10, step=1, description="", **_compact)

def _nov_label(txt):
    return widgets.HTML(f'<div style="font-size:9px;margin-top:2px;margin-bottom:0;line-height:1.2;color:#1565c0">{txt}</div>')

NOV_FACTORY = {"novelty_method": "spectral", "hop": 256, "gamma": 100.0, "local_avg": 10, "zoom_enabled": False, "zoom_level": 1}
nov_defaults = {
    "novelty_method": widgets.Dropdown(options=["energy", "spectral", "phase", "complex"], value=NOV_FACTORY["novelty_method"], **_compact),
    "hop": widgets.Dropdown(options=HOP_OPTIONS, value=NOV_FACTORY["hop"], **_compact),
    "gamma": widgets.Dropdown(options=GAMMA_OPTIONS, value=NOV_FACTORY["gamma"], **_compact),
    "local_avg": widgets.Dropdown(options=LOCAL_M_OPTIONS, value=NOV_FACTORY["local_avg"], **_compact),
    "zoom_enabled": widgets.Checkbox(value=NOV_FACTORY["zoom_enabled"], **_compact),
    "zoom_level": widgets.IntSlider(value=NOV_FACTORY["zoom_level"], min=1, max=10, **_compact),
}

_nov_state = {}

def _nov_sync_params(method):
    d = NOVELTY_DEFAULTS.get(method, NOVELTY_DEFAULTS["spectral"])
    nov_w_hop.value = d["H"] if d["H"] in [o[1] for o in HOP_OPTIONS] else 256
    g = d.get("gamma")
    nov_w_gamma.value = g if g is not None and g in [o[1] for o in GAMMA_OPTIONS] else 100.0
    m = d.get("M", 0)
    nov_w_local_avg.value = m if m in [o[1] for o in LOCAL_M_OPTIONS] else 10

nov_w_method.observe(lambda c: _nov_sync_params(c["new"]) if c["name"] == "value" else None, names="value")

def nov_w_do_recompute(_):
    import time
    t0 = time.perf_counter()
    nm = nov_w_method.value
    N = NOVELTY_DEFAULTS[nm]["N"]
    gamma_val = nov_w_gamma.value if NOVELTY_DEFAULTS[nm].get("gamma") is not None else None
    nov, nov_100 = get_novelty_for_method(novelty_cache, x, sr, nm, N, nov_w_hop.value, gamma_val, nov_w_local_avg.value)
    t_compute = time.perf_counter() - t0
    _nov_state.update({"nov_100": nov_100, "nm": nm})
    nov_w_redraw(0)
    with nov_w_status_out:
        nov_w_status_out.clear_output()
        print(f"Novelty: {t_compute:.3f}s")

def nov_w_redraw(center_time=None, update_audio=True):
    if not _nov_state:
        return
    half = zoom_level_to_half_sec(nov_w_zoom_level.value, duration_s) if nov_w_zoom_enabled.value else duration_s
    xlo = max(0, (center_time or 0) - half) if nov_w_zoom_enabled.value and center_time is not None else 0
    xhi = min(duration_s, (center_time or 0) + half) if nov_w_zoom_enabled.value and center_time is not None else duration_s
    if not nov_w_zoom_enabled.value:
        xlo, xhi = 0, duration_s
    with nov_w_plot_out:
        nov_w_plot_out.clear_output(wait=True)
        fig, axes = plt.subplots(2, 1, figsize=(12, 4), sharex=True)
        t_wav = np.arange(len(x)) / sr
        axes[0].plot(t_wav, x, color="#888", linewidth=0.5)
        axes[0].set_ylabel("Amp")
        axes[0].set_title("Waveform")
        axes[0].set_xlim(xlo, xhi)
        fig.text(0.5, 0.52, "Hop | Gamma | Local M", ha="center", fontsize=7, color="#666")
        t_nov = np.arange(len(_nov_state["nov_100"])) / FS_NOV
        axes[1].plot(t_nov, _nov_state["nov_100"], color="#333", linewidth=0.7)
        axes[1].set_ylabel("Novelty")
        axes[1].set_title(f"Novelty ({_nov_state['nm']})")
        axes[1].set_xlim(xlo, xhi)
        plt.tight_layout()
        plt.show()
    if update_audio:
        with nov_w_audio_out:
            nov_w_audio_out.clear_output(wait=True)
            h = build_audio_with_waveform_html(x, x, sr, max_duration_sec=max(60, duration_s))
            display(HTML('<div id="nov-w-audio-container">' + h + '</div>'))
            display(Javascript("(function(){ var c = document.getElementById('nov-w-audio-container'); var a = c ? c.querySelector('audio') : null; if (!a) return; var last = -1; a.addEventListener('timeupdate', function(){ var t = a.currentTime; if (Math.abs(t - last) < 0.25) return; last = t; if (typeof Jupyter !== 'undefined' && Jupyter.notebook && Jupyter.notebook.kernel) Jupyter.notebook.kernel.execute('nov_w_zoom_from_playhead(' + t + ')'); }); })();"))
def nov_w_zoom_from_playhead(t):
    if nov_w_zoom_enabled.value and t is not None:
        nov_w_redraw(float(t), update_audio=False)

nov_w_zoom_enabled.observe(lambda _: nov_w_redraw(0) if _nov_state else None, names="value")
nov_w_zoom_level.observe(lambda _: nov_w_redraw(0) if _nov_state else None, names="value")
nov_w_method.observe(lambda _: nov_w_do_recompute(None), names="value")
nov_w_hop.observe(lambda _: nov_w_do_recompute(None), names="value")
nov_w_gamma.observe(lambda _: nov_w_do_recompute(None), names="value")
nov_w_local_avg.observe(lambda _: nov_w_do_recompute(None), names="value")

_nov_param_widgets = {"novelty_method": nov_w_method, "hop": nov_w_hop, "gamma": nov_w_gamma, "local_avg": nov_w_local_avg, "zoom_enabled": nov_w_zoom_enabled, "zoom_level": nov_w_zoom_level}
def nov_w_reset(_):
    for k, w in nov_defaults.items():
        if k in ("zoom_enabled", "zoom_level"): continue
        try:
            v = w.value
            if v is None: continue
            if k == "gamma": v = float(v)
            elif k in ("local_avg", "zoom_level"): v = int(v)
            _nov_param_widgets[k].value = v
        except (TypeError, ValueError): pass
    nov_w_do_recompute(None)
def nov_w_set_default(_):
    for k in nov_defaults:
        nov_defaults[k].value = _nov_param_widgets[k].value
def nov_w_factory(_):
    for k, w in nov_defaults.items(): w.value = NOV_FACTORY[k]
    nov_w_reset(None)

nov_w_recompute_btn = widgets.Button(description="Recompute", button_style="primary")
nov_w_recompute_btn.on_click(nov_w_do_recompute)
nov_w_status_out = widgets.Output()
nov_w_plot_out = widgets.Output()
nov_w_audio_out = widgets.Output()

nov_w_controls = widgets.VBox([
    widgets.HTML("<b>Novelty</b>"), nov_w_method, _nov_label("Hop"), nov_w_hop, _nov_label("Gamma"), nov_w_gamma, _nov_label("Local M"), nov_w_local_avg,
    widgets.HTML("<b>Zoom</b>"), nov_w_zoom_enabled, _nov_label("Zoom (1=out, 10=in)"), nov_w_zoom_level,
    nov_w_recompute_btn,
    widgets.HTML("<b>Defaults</b>"), _nov_label("Novelty"), nov_defaults["novelty_method"], _nov_label("Hop"), nov_defaults["hop"], _nov_label("Gamma"), nov_defaults["gamma"], _nov_label("Local M"), nov_defaults["local_avg"],
    _nov_label("Zoom on"), nov_defaults["zoom_enabled"], _nov_label("Zoom level"), nov_defaults["zoom_level"],
    widgets.HBox([widgets.Button(description="RESET"), widgets.Button(description="SET TO DEFAULT")]), widgets.Button(description="Reset to factory", button_style="warning"),
], layout=widgets.Layout(width="280px"))
nov_w_controls.children[-2].children[0].on_click(nov_w_reset)
nov_w_controls.children[-2].children[1].on_click(nov_w_set_default)
nov_w_controls.children[-1].on_click(nov_w_factory)

nov_w_outputs = widgets.VBox([nov_w_status_out, nov_w_plot_out, widgets.HTML("<b>Audio</b>"), nov_w_audio_out], layout=widgets.Layout(flex="1"))
nov_widget_box = widgets.HBox([nov_w_controls, nov_w_outputs], layout=widgets.Layout(width="100%"))
display(nov_widget_box)
nov_w_do_recompute(None)